In [1]:
from molecular_unfolding.utility.MoleculeParser import MoleculeData
from molecular_unfolding.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

2022-12-27 09:16:06,012 dwave.cloud INFO MainThread Log level for 'dwave.cloud' namespace set to 0


ModuleNotFoundError: No module named 'py3Dmol'

In [ ]:
# prepare base container
# install py3Dmol

In [ ]:
# upload data to s3

In [59]:
import copy

c = copy.deepcopy

def get_key(single_dict):
    for k in single_dict.keys():
        return k

def parse_params(params_list, hp, hp_list):
    params = params_list[0]
    k = get_key(params)
    ps = params[k]
    for p in ps:
        hp[k] = p
        if len(params_list) == 1:
            hp_list.append(c(hp))
        else:
            parse_params(params_list[1:], hp, hp_list)

In [63]:
experiments_params =  {
    "version": "1",
    "params": [
        {"M": [1, 2, 3, 4, 5]},
        {"D": [8]},
        {"shots": [10000]},
        {"device": [{"qc": None, "cc": 'ml.m5.large'}]}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

In [64]:
hybrid_job_params

[{'M': 1, 'D': 8, 'shots': 10000, 'device': {'qc': None, 'cc': 'ml.m5.large'}},
 {'M': 2, 'D': 8, 'shots': 10000, 'device': {'qc': None, 'cc': 'ml.m5.large'}},
 {'M': 3, 'D': 8, 'shots': 10000, 'device': {'qc': None, 'cc': 'ml.m5.large'}},
 {'M': 4, 'D': 8, 'shots': 10000, 'device': {'qc': None, 'cc': 'ml.m5.large'}},
 {'M': 5, 'D': 8, 'shots': 10000, 'device': {'qc': None, 'cc': 'ml.m5.large'}}]

In [ ]:
# Pick 'autograd', 'tf', or 'torch'. Autograd is base PennyLane
def define_hyperparameters(interface):
    hyperparameters = {
        # Number of tasks per iteration = 2 * (num_nodes + num_edges) * p + 1
        "p": "2",
        "seed": str(seed),
        # Maximum number of simultaneous tasks allowed
        "max_parallel": "10",
        # Number of total optimization iterations, including those from previous checkpoint (if any)
        "num_iterations": "5",
        # Step size / learning rate for gradient descent
        "stepsize": "0.1",
        # Shots for each circuit execution
        "shots": "1000",
        "interface": interface,
    }
    return hyperparameters

In [ ]:
# parse evaluation parameters and trigger hybrid jobs:
hybrid_job_arns = []

jobs = []
names = []

task_name = 'molecular_unfolding'

job_params

for job_param in hybrid_job_params:
    print(f"Creating {task_name} with M {M} and D {D}")
    name = f"{task_name}-M-{M}-D-{D}-" + str(int(time.time()))
    hyperparams = {
        "M": str(M), 
        "D": str(D),   
        "local": "False"
    }

    tmp_job = AwsQuantumJob.create(
        device=device,
        source_module="qcbm",
        entry_point="qcbm.qcbm_job:main",
        job_name=name,
        hyperparameters=hyperparams,
        input_data="data.npy",
        wait_until_complete=False,
    )
    jobs.append(tmp_job)
    names.append(name)

In [4]:
!cp -r utility/ molecular_unfolding/